In [36]:
###
### SORT ACCORDING TO COLUMN
###

import sys, csv ,operator
data = csv.reader(open('import_data/publishers.csv'))
sortedlist = sorted(data, key=operator.itemgetter(1))    # 1 specifies according to first column we want to sort
#now write the sorte result into new CSV file
with open("import_data/publishers_sorted.csv", "w") as f:
  fileWriter = csv.writer(f, delimiter=',')
  for row in sortedlist:
      fileWriter.writerow(row)



In [42]:
###
### DEDUPLICATE DATA (check manually at the end)
###

reader=csv.reader(open('import_data/publishers_sorted.csv', 'r'), delimiter=',')
writer=csv.writer(open('import_data/publishers_sorted_distinct.csv', 'w'), delimiter=',')

entries = set()

for row in reader:
   key = (row[0], row[1]) # instead of just the last name

   if key not in entries:
      writer.writerow(row)
      entries.add(key)

In [42]:
###
### READ THE CSV
###

import csv

f = open('import_data/publishers_sorted_distinct_manuallychecked.csv')
csv_f = csv.reader(f)   
data = []

for row in csv_f: 
   data.append(row)
f.close()

## TEST COSA C'È NELLE LINEE SPECIFICATE
## print(data[1:3])

    

In [46]:
###
### READ THE CSV
###

import csv

f = open('import_data/publishers_sorted_distinct_manuallychecked.csv')
o = open('xml/publishers.xml', 'w')

csv_f = csv.reader(f)   
data = []

for row in csv_f: 
   data.append(row)
f.close()

## test cosa c'è nelle linee specificate
## print(data[1:3])



###
### TRANSFORM INTO XML FOR BULK IMPORT INTO KNORA
###


## FUNCTION THAT CREATES ITEM FROM CSV
def convert_row(row):
    ## create id with first or first and last word of the publisher's name
    word_list = row[1].replace("'", ' ') ## replace accent with space
    word_list = word_list.replace(",", '').split()  ## replace comma with nothing and split words
    if (word_list[0] == "La") or (word_list[0] == "L") or (word_list[0] == "Les") or (word_list[0] == "Le"): ## if the first word is ...
        if (word_list[1] == word_list[-1]): ## if the second and the last words are equal
            first_last_word = word_list[1]  ## take the second
        else:
            first_last_word = word_list[1]+'_'+word_list[-1]  ## otherwise take the second and the last
    else:    ## if the first word is NOT ...
        if (word_list[0] == word_list[-1]): ## if the first and the last words are equal
            first_last_word = word_list[0]  ## take the first
        else:
            first_last_word = word_list[0]+'_'+word_list[-1]   ## otherwise take the first and the last
    
    ## create item from csv
    return """
    <p0112-roud-oeuvres:Publisher id="edi_%s">
        <knoraXmlImport:label>edi_%s</knoraXmlImport:label>
        <p0112-roud-oeuvres:publisherHasLocation knoraType="richtext_value">%s</p0112-roud-oeuvres:publisherHasLocation>
        <p0112-roud-oeuvres:publisherHasName knoraType="richtext_value">%s</p0112-roud-oeuvres:publisherHasName>
    </p0112-roud-oeuvres:Publisher>
    """ % (first_last_word, row[1], row[0], row[1])


## WRITE DECLARATIONS AND BEGINNING OF THE XML FILE -adding line breaks for readibility-
o.write('<?xml version="1.0" encoding="UTF-8"?>'+'\n'+'<knoraXmlImport:resources xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"'+'\n'+'xsi:schemaLocation="http://api.knora.org/ontology/knoraXmlImport/v1# ../p0112-roud-oeuvres-xml-schemas/p0112-roud-oeuvres.xsd" '+'\n'+' xmlns="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" '+'\n'+' xmlns:knoraXmlImport="http://api.knora.org/ontology/knoraXmlImport/v1#" '+'\n'+' xmlns:p0112-roud-oeuvres="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#">'+'\n\n')

## WRITE ITEMS TO FILE, LOOP ON THE FUNCTION convert_row
o.write('\n'.join([convert_row(row) for row in data[1:]]))

## WRITE END OF THE XML FILE
o.write('</knoraXmlImport:resources>')

o.close

<function TextIOWrapper.close>